<a href="https://colab.research.google.com/github/8MeTools/ButtonTextReColorizer/blob/main/ButtonTextReColorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Googleドライブのマウント**
Googleドライブにアクセスして、ファイルの読み書きなどを行うために必要なコードです。実行した後、アクセスを許可してください。\
なお、このノートブックでは指定されたディレクトリ以外のファイル(ご自身の写真やドキュメントデータなど)を読み書きすることはありませんので、ご安心ください。

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 環境の準備
以下のセルを実行すると、必要なツールの導入などを行います。

**2025/11/23 0時ごろ(日本時間)にアップデートを行いました。ファイルの変更を行うために以下のコードを実行してください。**

In [2]:
# Googleドライブのマイドライブへ移動.
%cd /content/drive/MyDrive
#マイドライブ直下に8MeToolsフォルダを生成.
!mkdir -p ./8MeTools/BTRC
#8MeToolsフォルダに移動
%cd /content/drive/MyDrive/8MeTools
#8MeToolsフォルダ内に必要ファイルをダウンロード.
!git clone https://github.com/8MeTools/ButtonTextReColorizer.git
#フォルダ名のリネーム.
!mv "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"* "/content/drive/MyDrive/8MeTools/BTRC/"
#不要データの削除(GitHub上では必要ですが、作業においては必要のないものです)
!rm -r "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLYT/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/.gitkeep
# ファイル,ディレクトリが存在しているかどうかを確認する
%ls /content/drive/MyDrive/8MeTools/BTRC -l

/content/drive/MyDrive
/content/drive/MyDrive/8MeTools
Cloning into 'ButtonTextReColorizer'...
remote: Enumerating objects: 724, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 724 (delta 64), reused 72 (delta 49), pack-reused 613 (from 1)
Receiving objects: 100% (724/724), 316.48 KiB | 3.01 MiB/s, done.
Resolving deltas: 100% (489/489), done.
Updating files: 100% (590/590), done.
total 55
drwx------ 4 root root  4096 Nov 22 14:38 Assets/
-rw------- 1 root root 34673 Nov 22 14:38 ButtonTextReColorizer.ipynb
drwx------ 2 root root  4096 Nov 22 14:38 EditedBRLAN/
drwx------ 2 root root  4096 Nov 22 14:38 EditedBRLYT/
-rw------- 1 root root  1064 Nov 22 14:38 LICENSE
-rw------- 1 root root  3438 Nov 22 14:38 README.md
drwx------ 4 root root  4096 Nov 22 14:38 tmp/


In [ ]:
%cd /content/drive/MyDrive/8MeTools/ButtonTextReColorizer
!git clone https://github.com/stblr/wuj5.git
!pip install colour colr
!pip install json5
!ls -l

/content
Cloning into 'wuj5'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 171 (delta 58), reused 72 (delta 32), pack-reused 71 (from 1)
Receiving objects: 100% (171/171), 78.21 KiB | 3.72 MiB/s, done.
Resolving deltas: 100% (95/95), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for colr: filename=Colr-0.9.1-py3-none-any.whl size=78225 sha256=1a3d85707569b5d22fb182aca683c94a61a36368f88de53d2033dae082c38795
  Stored in directory: /root/.cache/pip/wheels/dd/9e/8a/9c47b62778b63ba07ef8ce256aa2f59435190d2bd148e7849c
Successfully built colr
/content/wuj5
total 92
-rw-r--r-- 1 root root 10175 Nov 22 14:38 bmg.py
-rw-r--r-- 1 root root  4875 Nov 22 14:38 brctr.py
-rw-r--r-- 1 root root 12666 Nov 22 14:38 brlan.py
-rw-r--r-- 1 root root 21882 Nov 22 14:38 brlyt.py
-rw-r--r-- 1 root root  7066

In [ ]:
!pwd

/content/wuj5


In [ ]:
from btrc import BRLYT_JSON5_DIR, BRLAN_JSON5_DIR, WUJ5_SCRIPT, EDITED_BRLYT_DIR, EDITED_BRLAN_DIR
from btrc.json5_io import list_json5_files, read_json5, write_json5
from btrc.brlyt import apply_tev_colors
from btrc.brlan import update_tev_colors, select_color_rule
from btrc.encode import encode_json5_files
from btrc.cleanup import remove_json5_files, move_all_files
from btrc.colors import run_color_input_flow, get_outline_color_from_user

In [ ]:
color_map = run_color_input_flow()

In [ ]:
default_free_text = (220, 220, 220)
default_select_text = (255, 255, 255)
free_outline = get_outline_color_from_user("free", "#282828")
select_outline = get_outline_color_from_user("select", "#787878")
text_free_colors = (default_free_text, free_outline)
text_select_colors = (default_select_text, select_outline)

In [ ]:
import os
import shutil

os.makedirs(BRLYT_JSON5_DIR, exist_ok=True)
os.makedirs(BRLAN_JSON5_DIR, exist_ok=True)

def copy_all(src_dir, dst_dir):
    for root, _, files in os.walk(src_dir):
        rel = os.path.relpath(root, src_dir)
        dst_root = os.path.join(dst_dir, rel) if rel != "." else dst_dir
        os.makedirs(dst_root, exist_ok=True)
        for f in files:
            src = os.path.join(root, f)
            dst = os.path.join(dst_root, f)
            shutil.copy2(src, dst)

copy_all(str(BRLYT_JSON5_DIR).replace("tmp/BRLYT", "Assets/BRLYT"), str(BRLYT_JSON5_DIR))
copy_all(str(BRLAN_JSON5_DIR).replace("tmp/BRLAN", "Assets/BRLAN"), str(BRLAN_JSON5_DIR))

In [ ]:
brlyt_files = list_json5_files(BRLYT_JSON5_DIR)
for path in brlyt_files:
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    new_text = apply_tev_colors(text, color_map)
    if new_text is not None:
        with open(path, "w", encoding="utf-8") as f:
            f.write(new_text)

In [ ]:
brlan_files = list_json5_files(BRLAN_JSON5_DIR)
for path in brlan_files:
    data = read_json5(path)
    rule = select_color_rule(path, text_free_colors, text_select_colors)
    if rule is None:
        continue
    (start_outline, start_text), (end_outline, end_text) = rule
    updated = update_tev_colors(data, start_outline, start_text, end_outline, end_text)
    write_json5(path, updated)

In [ ]:
encode_json5_files(brlyt_files + brlan_files, WUJ5_SCRIPT)
remove_json5_files(brlyt_files + brlan_files)

In [ ]:
import os
os.makedirs(EDITED_BRLYT_DIR, exist_ok=True)
os.makedirs(EDITED_BRLAN_DIR, exist_ok=True)
move_all_files(BRLYT_JSON5_DIR, EDITED_BRLYT_DIR)
move_all_files(BRLAN_JSON5_DIR, EDITED_BRLAN_DIR)

Googleドライブを開いてファイルをダウンロードしてください。お疲れ様でした。